In [ ]:
This notebook is from the https://learn.deeplearning.ai/langchain/lesson/3/memory and Prompt workshop

In [2]:
from langchain.document_loaders.github import GitHubIssuesLoader
import os

from dotenv import load_dotenv

load_dotenv()

GHA_TOKEN = os.getenv('GHA')

This section how to use prompt templates and output parser to get json info from a query

Note that StructuredOutputParser is only good for simple key/value pairs. Doesn't handle arrays or complex objects; 
for that you'd need Pydantic or Function parser

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers  import ResponseSchema, StructuredOutputParser

templateS = """
 Suggest a {style} product name  and tagline  for a company making {product}. {format_instructions}
"""

schema3 = ResponseSchema(name="name", description="""The name of the product
""", type="string")
schema4 = ResponseSchema(name="name", description="A catchy tagline for the product", type="string")
schemas = [schema3, schema4]
parser = StructuredOutputParser(response_schemas=schemas, only_json=True)

## we can set partial vars with the formatting instructions
prompt = ChatPromptTemplate.from_template(templateS, output_parser=parser, 
    partial_variables={"format_instructions": parser.get_format_instructions()})
llm = ChatOpenAI( model='gpt-4',temperature=0.5)

message = prompt.format_messages(style='serious', product='car')
resp=llm(message)

parser.parse(resp.content)

This next section shows 4 sorts of memory

In [79]:
## this appends to existing memory with each conversation
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains import ConversationChain

memory =ConversationBufferMemory()
## add 3 messages,  these are all remembered
memory.save_context(inputs={"input":  "Hi I'm Richard"}, outputs={"output":"hello from OpenaI"})
memory.save_context(inputs={"input":  "How are you"}, outputs={"output":"Just hanging"})
memory.save_context(inputs={"input":  "whats four squared"}, outputs={"output":"16"})
chain = ConversationChain(memory = memory, llm=llm)
resp = chain.predict(input ="What's my name again?")
print(resp)
memory.load_memory_variables({})





Your name is Richard.


{'history': "Human: Hi I'm Richard\nAI: hello from OpenaI\nHuman: How are you\nAI: Just hanging\nHuman: whats four squared\nAI: 16\nHuman: What's my name again?\nAI: Your name is Richard."}

Now let's use window of size 2. It'll forget my na,e

In [80]:

## this remembers up to last k exchanges
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory =ConversationBufferWindowMemory(k=2) # k = size of window
## add 3 messages, the 1st onw will be forgotten
memory.save_context(inputs={"input":  "Hi I'm Richard"}, outputs={"output":"hello from OpenaI"})
memory.save_context(inputs={"input":  "How are you"}, outputs={"output":"Just hanging"})
memory.save_context(inputs={"input":  "whats four squared"}, outputs={"output":"16"})
chain = ConversationChain(memory = memory, llm=llm)
resp = chain.predict(input ="What's my name again?")
print(resp)
memory.load_memory_variables({})

I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.


{'history': "Human: whats four squared\nAI: 16\nHuman: What's my name again?\nAI: I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."}

In [75]:
memory.load_memory_variables({})

{'history': "Human: What's four squared\nAI: 16\nHuman: what's the answer again?\nAI: The answer is 16."}

In [87]:

## this summarises the conversation history once the max size has been reached
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

memory =ConversationSummaryBufferMemory(llm=llm,max_token_limit=50) # k = size of window
## add 3 messages, the 1st onw will be forgotten
memory.save_context(inputs={"input":  "Hi I'm Richard"}, outputs={"output":"hello from OpenaI"})
memory.save_context(inputs={"input":  "What's my agenda today"}, outputs={"output":"You are going to the dentist at 4pm, then are off to London for the meeting"})
memory.save_context(inputs={"input":  "How are you"}, outputs={"output":"Just hanging"})
memory.save_context(inputs={"input":  "whats four squared"}, outputs={"output":"16"})
chain = ConversationChain(memory = memory, llm=llm)
resp = chain.predict(input ="What's my name again?")
print(resp)
memory.load_memory_variables({})## the system role includes a summary of the conversation thus far

Your name is Richard.


{'history': "System: Richard introduces himself to the AI from OpenAI and asks about his agenda for the day. The AI informs him that he has a dentist appointment at 4pm, followed by a meeting in London.\nHuman: How are you\nAI: Just hanging\nHuman: whats four squared\nAI: 16\nHuman: What's my name again?\nAI: Your name is Richard."}

In [98]:
## this summarises the conversation history once the max size has been reached
from langchain.memory import ConversationEntityMemory
from langchain.chains import ConversationChain
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

memory =ConversationEntityMemory(llm=llm)

memory.save_context(inputs={"input":  "Hi I'm Richard, a developer"}, outputs={"output":"hello from OpenaI"})
memory.save_context(inputs={"input":  "What's my agenda today"}, outputs={"output":"You are going to the dentist at 4pm, then are off to London for the meeting"})
memory.save_context(inputs={"input":  "How are you"}, outputs={"output":"Just hanging around in Edinburgh"})
memory.save_context(inputs={"input":  "whats four squared"}, outputs={"output":"16"})
chain = ConversationChain(memory = memory, llm=llm, prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,verbose=True)
resp = chain.predict(input="Deven and Sam are working on a hackathon project")
print(resp)
memory.entity_cache# the system role includes a summary of the conversation thus far



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

['Deven', 'Sam', 'Hackathon']

In [100]:
chain.predict(input="they are working on an alternative to Github")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's an ambitious project! Building an alternative to Github would involve creating a platform for version control and collaboration, which is quite complex. It could include features like issue tracking, pull requests, code reviewing, and more. Do they have any specific features or improvements in mind that they want to implement in their alternative?"

In [102]:
memory.entity_store.store

{'Deven': 'Deven is currently working on a hackathon project with Sam.',
 'Sam': 'Sam is currently working on a hackathon project with Deven.',
 'Hackathon': 'A hackathon is an event where Deven and Sam are currently working on a project.',
 'Github': 'Github is a platform for version control and collaboration, featuring issue tracking, pull requests, and code reviewing.'}